In [9]:
# DRIMEL https://drimel.com.ar
import requests
import pandas as pd
from bs4 import BeautifulSoup
import credentials
import re

def scrape_drimel(login_url, urls):
    session = requests.Session()

    payload = {'email': credentials.email_drimel, 'password': credentials.password_drimel}

    response = session.post(login_url, data=payload)

    if response.status_code == 200:
        print('Inicio de sesión exitoso')
    else:
        print('Inicio de sesión fallido')


    marcas = ['Algabo', 'Duffy', 'Huggies','Babysec','Caricia','Estrella','Pampers',
              'Candy','Doncella','Deyse','Johnsons','Kimbies','Upa']

    df = pd.DataFrame(columns=['name', 'price','marca','product_id', 'product_sku','url_id','MPN (Número de pieza del fabricante)'])        
    
    
    # Buscamos los elementos que contienen la información de los productos
    
    for extract_url in urls: 


        page = 1
        while True:
            url = extract_url + 'page/{}/'.format(page)

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            products = soup.find_all('div', class_='box-text box-text-products')
        
            if not products:
                break

            # Iteramos sobre los elementos encontrados y extraemos la información deseada
            for product in products:
                name = product.find('a', class_='woocommerce-LoopProduct-link').text.strip()
            
                price = product.find('span', class_='woocommerce-Price-amount').text.strip()[1:] # Quitamos el signo $
                price = price.replace('"', '').replace('.', '').split(',')[0] # Quitamos las comillas, la coma y lo que hay después del punto
                
                
                # Buscamos si el nombre del producto contiene alguna de las marcas
                marca = ''
                for m in marcas:
                    if m.lower() in name.lower():
                        marca = m
                        break
                # Obtenemos el código de barras y el código común del producto
                add_to_cart_button = product.find('a', class_='add_to_cart_button')
                product_id = add_to_cart_button.get('data-product_id', '')
                product_sku = add_to_cart_button.get('data-product_sku', '')

                url_id = re.sub('[^a-zA-Z0-9]+', '-', name).strip('-')
                url_id = url_id.replace(' ', '-')

                # Excluimos las filas que tengan como marca 'Algabo' o 'Upa'
                df = df[(df['marca'] != 'Algabo') & (df['marca'] != 'Upa')]

                MPN = 'Drimel'
                df.loc[len(df)] = [name, price, marca, product_id, product_sku, url_id, MPN]
            page += 1
                # Puedes hacer lo que necesites con la información obtenida, como imprimir o guardar en un archivo
    df = df.rename(columns={'name': 'Nombre', 'price': 'Precio',
                             'product_id': 'SKU', 'product_sku': 'Código de barras',
                               'url_id':'Identificador de URL', 'marca':'Marca'})

    df['SKU'] = df['SKU'].astype(int)
    df = df.sort_values('SKU', ascending=True)

    df.to_csv('../procesados/drimel.csv', index=False)
    print('Archivo guardado con exito')
    return 
        


In [10]:
scrape_drimel('https://drimel.com.ar/my-account/' , ['https://drimel.com.ar/categoria/panales/bebes/','https://drimel.com.ar/categoria/cuidado-del-bebe/'])

Inicio de sesión exitoso
Archivo guardado con exito
